In [ ]:
import json
from confluent_kafka import Consumer, Producer

# Налаштування Kafka
kafka_config = {
    "bootstrap_servers": ["77.81.230.104:9092"],
    "username": "admin",
    "password": "VawEzo1ikLtrA8Ug8THa",
    "security_protocol": "SASL_PLAINTEXT",
    "sasl_mechanism": "PLAIN",
}

SENSOR_TOPIC = "building_sensors_your_name"  # Замініть 'your_name' на ваш ідентифікатор
TEMPERATURE_ALERTS_TOPIC = "temperature_alerts_your_name"
HUMIDITY_ALERTS_TOPIC = "humidity_alerts_your_name"

# Ініціалізація консюмера Kafka
consumer = Consumer(
    {
        "bootstrap.servers": kafka_config["bootstrap_servers"],
        "group.id": "sensor_group",
        "auto.offset.reset": "earliest",
        "security.protocol": kafka_config["security_protocol"],
        "sasl.mechanism": kafka_config["sasl_mechanism"],
        "sasl.username": kafka_config["username"],
        "sasl.password": kafka_config["password"],
    }
)
consumer.subscribe([SENSOR_TOPIC])

# Ініціалізація продюсера Kafka для сповіщень
producer = Producer(
    {
        "bootstrap.servers": kafka_config["bootstrap_servers"],
        "security.protocol": kafka_config["security_protocol"],
        "sasl.mechanism": kafka_config["sasl_mechanism"],
        "sasl.username": kafka_config["username"],
        "sasl.password": kafka_config["password"],
    }
)


def send_alert(topic, alert):
    producer.produce(topic, value=json.dumps(alert).encode("utf-8"))
    producer.flush()


try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            continue
        if msg.error():
            print(f"Помилка: {msg.error()}")
            continue

        data = json.loads(msg.value().decode("utf-8"))
        sensor_id = data["sensor_id"]
        temperature = data["temperature"]
        humidity = data["humidity"]
        timestamp = data["timestamp"]

        # Перевірка температури
        if temperature > 40.0:
            alert = {
                "sensor_id": sensor_id,
                "timestamp": timestamp,
                "temperature": temperature,
                "message": "Перевищено допустимий рівень температури!",
            }
            send_alert(TEMPERATURE_ALERTS_TOPIC, alert)
            print(f"Відправлено сповіщення про температуру: {alert}")

        # Перевірка вологості
        if humidity > 80.0 or humidity < 20.0:
            alert = {
                "sensor_id": sensor_id,
                "timestamp": timestamp,
                "humidity": humidity,
                "message": "Вологість виходить за допустимі рамки!",
            }
            send_alert(HUMIDITY_ALERTS_TOPIC, alert)
            print(f"Відправлено сповіщення про вологість: {alert}")

except KeyboardInterrupt:
    print("Завершення роботи скрипту...")
finally:
    consumer.close()

ModuleNotFoundError: No module named 'kafka.vendor.six.moves'